In [ ]:
# import libraries
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image

In [ ]:
# load data
data = pd.read_csv("iodine_per_day.csv", sep=";")

In [ ]:
# preview data
data.head()

In [ ]:
# drop unused data
mask_keep = data["Inclusion"] == 1
data = data.loc[mask_keep, :].reset_index(drop=True)

In [ ]:
# format numbers from strings to float
columns = data.columns[5:]
for column in columns:
    data.loc[:, column] = [strnb.replace(",", ".") for strnb in data.loc[:, column]]
data[columns] = data[columns].astype("float")

In [ ]:
# make dafaframe for recommended iodine quantities
data_rec = pd.DataFrame()
data_rec["Enfants"] = [70, 130]
data_rec["Adultes"] = [145, 155]
data_rec["Femmes enceintes"] = [195, 205]

# make dafaframe for max iodine quantities
data_max = pd.DataFrame()
data_max["Enfants"] = [200, 500]
data_max["Adultes"] = [595, 605]
data_max["Femmes enceintes"] = [595, 605]

In [ ]:
fig = make_subplots(
    rows=2,
    cols=1,
    row_heights=[0.65, 0.35],
    vertical_spacing=0.05)
config = {'displayModeBar': True}

# plot seaweed quantities
columns = ['5 grammes d’algue', '2 grammes d’algue', '1 gramme d’algue']
marker_sizes = [64, 32, 16]
for i in range(len(columns)):
    fig.add_trace(
        go.Scatter(
            x=data.loc[:, columns[i]],
            y="""<span style="color:""" + data.loc[:, "Couleur"] + """"><b>""" + \
                data.loc[:, "Nom vernaculaire"] + "</b><br><i>" + data.loc[:, "Nom scientifique"] + """</i></span>""",
            name=columns[i] + " déshydratée",
            mode="markers",
            marker=dict(
                symbol="circle",
                size=marker_sizes[i],
                line=dict(width=3, color=data.loc[:, "Couleur"]),
                color="white"
            ),
            showlegend=False,
            hovertemplate="""Iode : %{x} μg<extra><b style="color:""" + data.loc[:, "Couleur"] + """">""" + \
                columns[i] + """ déshydratée</b></extra>""",
            hoverlabel=dict(
                font_size=14,
                font_color="white",
                bgcolor=data.loc[:, "Couleur"],
                bordercolor="white"
            ),
        ),
        row=1, col=1
    )

# add seaweed images
seaweeds = ["Dulse", "Wakamé", "Laitue de mer"]
image_sizes = [160, 80, 40]
for i in range(len(seaweeds)):
    for j in range(len(columns)):
        fig.add_layout_image(
            dict(
                source=Image.open(f"Illustration algue-{seaweeds[i]}.png"),
                xref="x",
                yref="y",
                xanchor="center",
                yanchor="middle",
                x=data.loc[i, columns[j]],
                y="""<span style="color:""" + data.loc[i, "Couleur"] + """"><b>""" + \
                        data.loc[i, "Nom vernaculaire"] + "</b><br><i>" + data.loc[i, "Nom scientifique"] + """</i></span>""",
                sizex=image_sizes[j],
                sizey=image_sizes[j],
                sizing="contain",
                opacity=1.0,
                layer="above",
                visible=True
            ),
        )

# plot recommended quantities
fig.add_trace(
    go.Bar(
        x=data_rec.loc[0, :] - data_rec.loc[1, :],
        base=data_rec.loc[1, :],
        y=data_rec.columns,
        marker=dict(
            cornerradius=10,
            color="#93C9F9"
        ),
        name="Quantité journalière recommandée",
        orientation="h",
        hoverinfo="skip",
        legendgroup="2",
    ),
    row=2,col=1
)

# plot max quantities
fig.add_trace(
    go.Bar(
        x=data_max.loc[0, :] - data_max.loc[1, :],
        base=data_max.loc[1, :],
        y=data_max.columns,
        marker=dict(
            cornerradius=10,
            color="#113972"
        ),
        name="Quantité journalière maximum",
        orientation="h",
        hoverinfo="skip"
    ),
    row=2, col=1
)

fig.update_layout(
    plot_bgcolor="white",
    font_family="Montserrat",
    barmode="stack",
    hovermode="closest",
    legend=dict(
        font_color="#113972",
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    ),
    xaxis1=dict(
        showgrid=True,
        showticklabels=False,
        tickvals=[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300],
        range=[0, 1310],
        gridcolor="#93C9F9"
    ),
    xaxis2=dict(
        color="#93C9F9",
        title=dict(
            text="Quantité d'iode (en μg par jour)",
            font=dict(size=13)
        ),
        showline=False,
        showgrid=True,
        showticklabels=True,
        tickangle=0,
        ticklabelstandoff=10,
        tickvals=[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300],
        range=[0, 1310],
        gridcolor="#93C9F9"
    ),
    yaxis1=dict(
        showline=True,
        linecolor="#93C9F9",
        tickangle=0,
        ticklabelstandoff=10,
        range=[-1.0, 3.0],
    ),
    yaxis2=dict(
        showline=True,
        color="#113972",
        linecolor="#93C9F9",
        tickangle=0,
        ticklabelstandoff=10,
        autorange="reversed"
    ),
    margin=dict(
        t=50,
        b=20,
        l=20,
        r=20
    ),
    modebar=dict(
        orientation="h",
    ),
    height=400,
    width=800
)

fig.show(config=config)

In [ ]:
fig.write_html("iodineperday.html", include_plotlyjs="cdn")